### Function definitions (Algorithms)

In [1]:
from itertools import combinations

def compute_closure(attributes, fds)->set:
    """
    Compute the closure of a set of attributes under a set of functional dependencies
    ---------------------------------------------------------------------------------
    attributes: a set of attributes
    fds: a list of functional dependencies (contains tuples of two sets. First set implies the second set)
    """
    closure = set(attributes)

    changed = True
    while changed:
        changed = False
        for fd in fds:
            if fd[0].issubset(closure) and not fd[1].issubset(closure):
                closure.update(fd[1])
                changed = True

    return closure

def compute_all_closures(attributes, fds)->dict:
    """
    Compute the closure of all possible subsets of a set of attributes
    ------------------------------------------------------------------
    attributes: a set of attributes
    fds: a list of functional dependencies (contains tuples of two sets. First set implies the second set)
    """
    all_closures = {}
    for r in range(1, len(attributes) + 1):
        for subset in combinations(attributes, r):
            subset_closure = compute_closure(set(subset), fds)
            all_closures[tuple(subset)] = subset_closure
    return all_closures

def compute_candidate_keys(closure_set, attributes)->list:
    """
    Compute the candidate keys of a set of attributes
    -------------------------------------------------
    closure_set: a dictionary of all closures
    attributes: a set of attributes
    """
    super_keys = []
    for i in closure_set:
        if set(closure_set[i]) == set(attributes):
            super_keys.append(i)
    candidate_keys = []
    for j in super_keys:
        flag = False
        for i in super_keys:
            if set(i) != set(j):
                if set(i).issubset(set(j)):
                    flag = True
        if flag == False:
            candidate_keys.append(j)
    return candidate_keys

def find_prime_attributes(candidate_keys)->set:
    """
    Find the prime attributes of a set of candidate keys
    ----------------------------------------------------
    candidate_keys: a list of candidate keys
    """
    prime_attributes = set()
    for key in candidate_keys:
        prime_attributes.update(key)
    return prime_attributes

def compute_single_covers(attributes, fds)->dict:
    """
    Compute the closure of each attribute in a set of attributes
    ------------------------------------------------------------
    attributes: a set of attributes
    fds: a list of functional dependencies (contains tuples of two sets. First set implies the second set)
    """
    all_closures = {}
    for a in attributes:
        subset_closure = compute_closure(a, fds)
        all_closures[a] = subset_closure
    return all_closures

In [35]:
## Minimal cover computation

def decompose_fds(fds)->list:
    """Decompose each FD so that the RHS contains only one attribute.
    For example, the FD {A} -> {B, C} will be decomposed into {A} -> {B} and {A} -> {C}.
    ------------------------------------------------------------------------------------
    fds: a list of functional dependencies (contains tuples of two sets. First set implies the second set)
    """
    decomposed_fds = []
    for lhs, rhs in fds:
        for attr in rhs:
            decomposed_fds.append((lhs, {attr}))
    return decomposed_fds

def remove_trivial_dependencies(fds)->list:
    """Remove trivial FDs of the form A -> A.
    -----------------------------------------
    fds: a list of functional dependencies (contains tuples of two sets. First set implies the second set)
    """
    return [(lhs, rhs) for lhs, rhs in fds if lhs != rhs]

def remove_redundant_dependencies(fds, attributes)->list:
    """Remove redundant FDs by checking if we can infer a FD from others.
    ---------------------------------------------------------------------
    fds: a list of functional dependencies (contains tuples of two sets. First set implies the second set)
    attributes: a set of attributes"""
    non_redundant_fds = []
    
    for i, (lhs, rhs) in enumerate(fds):
        remaining_fds = fds[:i] + fds[i+1:]
        closure_lhs = compute_closure(lhs, remaining_fds)
        
        if not rhs.issubset(closure_lhs):
            non_redundant_fds.append((lhs, rhs))
    
    return non_redundant_fds

def merge_fds(fds)->list:
    """Merge FDs with the same LHS back together.
    --------------------------------------------
    fds: a list of functional dependencies (contains tuples of two sets. First set implies the second set)
    """
    merged_fds = {}
    for lhs, rhs in fds:
        lhs = tuple(lhs)
        if lhs in merged_fds:
            merged_fds[lhs].update(rhs)
        else:
            merged_fds[lhs] = set(rhs)
    
    return [(set(lhs), rhs) for lhs, rhs in merged_fds.items()]

def minimal_cover(attributes, fds)->list:
    """Find the minimal cover of a set of FDs.
    -----------------------------------------
    attributes: a set of attributes
    fds: a list of functional dependencies (contains tuples of two sets. First set implies the second set
    """
    decomposed_fds = decompose_fds(fds)

    simplified_fds = remove_trivial_dependencies(decomposed_fds)
    simplified_fds = remove_redundant_dependencies(simplified_fds, attributes)
    
    minimal_fds = merge_fds(simplified_fds)
    
    return minimal_fds

### Example usage

In [72]:
attributes = {'A', 'B', 'C', 'D', 'E'}
fds = [
    ({'A'}, {'A', 'B', 'C'}),
    ({'A', 'B'}, {'A'}),
    ({'B', 'C'}, {'A', 'D'}),
    ({'B'}, {'A', 'B'}),
    ({'C'}, {'D'})
]

In [73]:
all_closures = compute_all_closures(attributes, fds)

In [74]:
for k in all_closures:
    print('{k}+ = {v}'.format(k=set(k), v=all_closures[k]))

{'A'}+ = {'B', 'C', 'A', 'D'}
{'B'}+ = {'B', 'C', 'A', 'D'}
{'C'}+ = {'C', 'D'}
{'D'}+ = {'D'}
{'E'}+ = {'E'}
{'B', 'A'}+ = {'B', 'C', 'A', 'D'}
{'C', 'A'}+ = {'B', 'C', 'A', 'D'}
{'D', 'A'}+ = {'A', 'B', 'C', 'D'}
{'E', 'A'}+ = {'B', 'C', 'A', 'E', 'D'}
{'B', 'C'}+ = {'A', 'B', 'C', 'D'}
{'B', 'D'}+ = {'B', 'C', 'A', 'D'}
{'E', 'B'}+ = {'B', 'C', 'A', 'E', 'D'}
{'C', 'D'}+ = {'C', 'D'}
{'E', 'C'}+ = {'E', 'C', 'D'}
{'E', 'D'}+ = {'E', 'D'}
{'B', 'C', 'A'}+ = {'B', 'C', 'A', 'D'}
{'B', 'D', 'A'}+ = {'A', 'B', 'C', 'D'}
{'E', 'B', 'A'}+ = {'B', 'C', 'A', 'E', 'D'}
{'C', 'D', 'A'}+ = {'A', 'B', 'C', 'D'}
{'E', 'C', 'A'}+ = {'B', 'C', 'A', 'E', 'D'}
{'E', 'D', 'A'}+ = {'A', 'B', 'C', 'D', 'E'}
{'B', 'C', 'D'}+ = {'A', 'B', 'C', 'D'}
{'E', 'B', 'C'}+ = {'A', 'B', 'C', 'D', 'E'}
{'E', 'B', 'D'}+ = {'A', 'B', 'C', 'D', 'E'}
{'E', 'C', 'D'}+ = {'E', 'C', 'D'}
{'B', 'C', 'D', 'A'}+ = {'B', 'C', 'D', 'A'}
{'E', 'B', 'C', 'A'}+ = {'B', 'C', 'A', 'E', 'D'}
{'E', 'B', 'D', 'A'}+ = {'A', 'B', 'C', 

In [75]:
compute_candidate_keys(all_closures, attributes)

[('A', 'E'), ('B', 'E')]

In [76]:
find_prime_attributes(compute_candidate_keys(all_closures, attributes))

{'A', 'B', 'E'}

In [77]:
minimal_fds = minimal_cover(attributes, fds)
for lhs, rhs in minimal_fds:
    print(f"{lhs} -> {rhs}")

{'A'} -> {'B', 'C'}
{'B'} -> {'A'}
{'C'} -> {'D'}
